In [32]:
from sklearnex import patch_sklearn
patch_sklearn()

import pandas as pd
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer
import time
from sklearn.preprocessing import LabelEncoder

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [33]:
train = pd.read_csv("train_with_seasonal.csv")
test = pd.read_csv("test_filtered2.csv")

In [34]:
test.head()

,ID,date,cluster_id,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant
0,cluster_1_2022-01-01,2022-01-01,cluster_1,15.2,7.9,21902.986667,29784.506667,22.2,172.732245
1,cluster_2_2022-01-01,2022-01-01,cluster_2,15.9,4.1,23550.080000,30647.540000,22.7,163.972679
2,cluster_3_2022-01-01,2022-01-01,cluster_3,18.6,4.9,24179.643333,31545.103333,15.5,198.645137
3,cluster_4_2022-01-01,2022-01-01,cluster_4,13.3,3.9,23839.203333,30766.446667,16.1,183.517316
4,cluster_1_2022-01-02,2022-01-02,cluster_1,14.1,8.1,21018.193333,29847.110000,34.4,223.335840


In [35]:
train.head()

,ID,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant,month,dayofweek,quarter,day,weekofyear
0,cluster_1_2014-01-01,cluster_1,358.032,10.8,4.2,53.003333,29787.533333,40.6,186.000000,1,2,1,1,1
1,cluster_2_2014-01-01,cluster_2,548.247,12.2,4.3,8195.656667,30650.350000,36.0,170.591118,1,2,1,1,1
2,cluster_3_2014-01-01,cluster_3,758.303,12.9,-0.8,16305.260000,31547.686667,20.9,159.467752,1,2,1,1,1
3,cluster_4_2014-01-01,cluster_4,1072.077,10.8,4.7,9224.803333,30769.220000,34.7,184.339753,1,2,1,1,1
4,cluster_1_2014-01-02,cluster_1,386.908,10.7,7.0,22372.000000,29850.226667,34.3,210.333465,1,3,1,2,1


In [36]:
test['date'] = pd.to_datetime(test['date'])

In [37]:
test_with_seasonal = test.copy()

In [38]:
test_with_seasonal["month"] = test_with_seasonal["date"].dt.month
test_with_seasonal["dayofweek"] = test_with_seasonal["date"].dt.dayofweek
test_with_seasonal["quarter"] = test_with_seasonal["date"].dt.quarter
test_with_seasonal["day"] = test_with_seasonal["date"].dt.day
test_with_seasonal["weekofyear"] = test_with_seasonal["date"].dt.isocalendar().week.astype(int)
test_with_seasonal = test_with_seasonal.drop(columns=["date"])

In [39]:
test_with_seasonal.to_csv("test_with_seasonal.csv", index=False)

In [40]:
test = pd.read_csv("test_with_seasonal.csv")

In [41]:
test.head()

,ID,cluster_id,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant,month,dayofweek,quarter,day,weekofyear
0,cluster_1_2022-01-01,cluster_1,15.2,7.9,21902.986667,29784.506667,22.2,172.732245,1,5,1,1,52
1,cluster_2_2022-01-01,cluster_2,15.9,4.1,23550.080000,30647.540000,22.7,163.972679,1,5,1,1,52
2,cluster_3_2022-01-01,cluster_3,18.6,4.9,24179.643333,31545.103333,15.5,198.645137,1,5,1,1,52
3,cluster_4_2022-01-01,cluster_4,13.3,3.9,23839.203333,30766.446667,16.1,183.517316,1,5,1,1,52
4,cluster_1_2022-01-02,cluster_1,14.1,8.1,21018.193333,29847.110000,34.4,223.335840,1,6,1,2,52


In [42]:
if 'cluster_id' in test.columns:
    le = LabelEncoder()
    test['cluster_id'] = le.fit_transform(test['cluster_id'])

In [43]:
if 'cluster_id' in train.columns:
    le = LabelEncoder()
    train['cluster_id'] = le.fit_transform(train['cluster_id'])

In [44]:
train.head()

,ID,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant,month,dayofweek,quarter,day,weekofyear
0,cluster_1_2014-01-01,0,358.032,10.8,4.2,53.003333,29787.533333,40.6,186.000000,1,2,1,1,1
1,cluster_2_2014-01-01,1,548.247,12.2,4.3,8195.656667,30650.350000,36.0,170.591118,1,2,1,1,1
2,cluster_3_2014-01-01,2,758.303,12.9,-0.8,16305.260000,31547.686667,20.9,159.467752,1,2,1,1,1
3,cluster_4_2014-01-01,3,1072.077,10.8,4.7,9224.803333,30769.220000,34.7,184.339753,1,2,1,1,1
4,cluster_1_2014-01-02,0,386.908,10.7,7.0,22372.000000,29850.226667,34.3,210.333465,1,3,1,2,1


In [45]:
test.head()

,ID,cluster_id,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant,month,dayofweek,quarter,day,weekofyear
0,cluster_1_2022-01-01,0,15.2,7.9,21902.986667,29784.506667,22.2,172.732245,1,5,1,1,52
1,cluster_2_2022-01-01,1,15.9,4.1,23550.080000,30647.540000,22.7,163.972679,1,5,1,1,52
2,cluster_3_2022-01-01,2,18.6,4.9,24179.643333,31545.103333,15.5,198.645137,1,5,1,1,52
3,cluster_4_2022-01-01,3,13.3,3.9,23839.203333,30766.446667,16.1,183.517316,1,5,1,1,52
4,cluster_1_2022-01-02,0,14.1,8.1,21018.193333,29847.110000,34.4,223.335840,1,6,1,2,52


In [48]:
train.to_csv('train_with_seasonal_cluster.csv', index=False)
test.to_csv('test_with_seasonal_cluster.csv', index=False)

In [17]:
target_col = "electricity_consumption"

In [24]:
def train_and_eval(df_input, label):
    X = df_input.drop(columns=[target_col])
    y = df_input[target_col]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    model = XGBRegressor(objective="reg:squarederror", n_estimators=300, learning_rate=0.1, max_depth=5)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"✅ {label} RMSE: {rmse:.4f}")

    return model  # return the trained model

In [25]:
train_cluster_date = train.drop(columns=["ID"])

In [26]:
train_cluster_date.head()

,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant,month,dayofweek,quarter,day,weekofyear
0,0,358.032,10.8,4.2,53.003333,29787.533333,40.6,186.000000,1,2,1,1,1
1,1,548.247,12.2,4.3,8195.656667,30650.350000,36.0,170.591118,1,2,1,1,1
2,2,758.303,12.9,-0.8,16305.260000,31547.686667,20.9,159.467752,1,2,1,1,1
3,3,1072.077,10.8,4.7,9224.803333,30769.220000,34.7,184.339753,1,2,1,1,1
4,0,386.908,10.7,7.0,22372.000000,29850.226667,34.3,210.333465,1,3,1,2,1


In [28]:
model = train_and_eval(train_cluster_date, "With seasonal and clusterid features")

✅ With seasonal and clusterid features RMSE: 28.5746


In [23]:
X_test = test.drop(columns=["ID"]) 

In [29]:
y_test_pred = model.predict(X_test)

In [31]:
submission = pd.DataFrame({
    "ID": test["ID"],
    "electricity_consumption": y_test_pred
})
submission.to_csv("submission7.csv", index=False)